In [10]:
%matplotlib inline

from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils
from scipy import io
import numpy as np
from sklearn.cross_validation import train_test_split
import string
import cv2
import os
import sys

import seaborn as sns
sns.set_style("white")

In [2]:
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        '/home/ert/Pobrane/rips/data_train',
        target_size=(150, 150),
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        '/home/ert/Pobrane/rips/data_valid',
        target_size=(150, 150),
        class_mode='categorical')

Found 1560 images belonging to 2 classes.
Found 320 images belonging to 2 classes.


In [14]:
def load_folder(folder):
    
    images = []
    i = 0
    for file in os.listdir(folder):
        image = cv2.imread(os.path.join(folder, file))
        if(image is not None):
            image = cv2.resize(image, (resolution_x, resolution_y), interpolation=cv2.INTER_LINEAR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images.append(image)
        if (i%10==0):
            print (".", end="")
        i+=1
    print ("")
    return np.array(images)

In [15]:
fobia_folder = "/home/ert/Pobrane/rips/data_train/reddit_sub_trypophobia"
normal_folder = "/home/ert/Pobrane/rips/data_train/reddit_sub_pics"
fobia_folder_val = "/home/ert/Pobrane/rips/data_valid/reddit_sub_trypophobia"
normal_folder_val = "/home/ert/Pobrane/rips/data_valid/reddit_sub_pics"
resolution_x = 150
resolution_y = 150
fobia_images = load_folder(fobia_folder) /255.
normal_images = load_folder(normal_folder) /255.
fobia_images_val = load_folder(fobia_folder_val) /255.
normal_images_val = load_folder(normal_folder_val) /255.

.............................................................................................................................................................................................

In [16]:
def image_to_X_Y(fobia_images, normal_images):
    X_train = [] 
    Y_train = [] # 1,0 -> fobia | 0,1 -> normal
    for img in fobia_images:
        X_train.append(img)
        Y_train.append([1,0])
    for img in normal_images:
        X_train.append(img)
        Y_train.append([0,1])
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    return X_train, Y_train

X_train, Y_train = image_to_X_Y(fobia_images, normal_images)
X_test, Y_test = image_to_X_Y(fobia_images_val, normal_images_val)

In [17]:
model = Sequential()
model.add(Convolution2D(8, 3, 3, border_mode='same',
          input_shape=(150, 150, 3)))
#model.add(Convolution2D(8, 3, 3, border_mode='same'))
#model.add(Convolution2D(16, 3, 3, border_mode='same'))
#model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Convolution2D(128, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(128, 3, 3, border_mode='same'))
model.add(Activation('relu'))
#model.add(Dropout(0.25))
model.add(GlobalAveragePooling2D())
#model.add(Flatten())
#for i in range(1):
#    model.add(Dense(128))
#    model.add(Activation('relu'))
#    #model.add(Dropout(0.1))
#for i in range(1):
#    model.add(Dense(32))
#    model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit_generator(
        train_generator,
        samples_per_epoch=1536,
        nb_epoch=20,
        validation_data=test_generator,
        nb_val_samples=320)

Epoch 1/20


KeyboardInterrupt: 

In [19]:
history = model.fit(X_train, Y_train,
                    nb_epoch=20,
                    batch_size=64,
                    validation_data=(X_test, Y_test))

Train on 1560 samples, validate on 320 samples
Epoch 1/20
  64/1560 [>.............................] - ETA: 174s - loss: 0.6931 - acc: 0.4688

KeyboardInterrupt: 

In [6]:
from keras_sequential_ascii import sequential_model_to_ascii_printout
sequential_model_to_ascii_printout(model)

      OPERATION           DATA DIMENSIONS   WEIGHTS(N)   WEIGHTS(%)

          Input   #####   (150, 150, 3)
  Convolution2D    \|/  -------------------       224     0.1%
                  #####   (150, 150, 8)
  Convolution2D    \|/  -------------------      2336     0.7%
           relu   #####   (150, 150, 32)
   MaxPooling2D   YYYYY -------------------         0     0.0%
                  #####   (75, 75, 32)
  Convolution2D    \|/  -------------------     18496     5.8%
           relu   #####   (75, 75, 64)
  Convolution2D    \|/  -------------------     36928    11.7%
           relu   #####   (75, 75, 64)
  Convolution2D    \|/  -------------------     36928    11.7%
           relu   #####   (75, 75, 64)
   MaxPooling2D   YYYYY -------------------         0     0.0%
                  #####   (18, 18, 64)
  Convolution2D    \|/  -------------------     73856    23.3%
           relu   #####   (18, 18, 128)
  Convolution2D    \|/  -------------------    147584    46.6%
        